# III. Введение в пространственный анализ (2)


На этой неделе мы продолжим занкомиться с разными инструментами пространственного анализа: научимся агрегировать атрибутивную информацию, познакомимся с Диаграммой Вороного и Индексом ближайшего соседа

Эти навыки помогут дополнить анализ исторической вспышки холеры в Лондоне (1854) – [второй проект](../projects/project2.ipynb) на нашем курсе.


0. Импортируем нужные библиотеки


In [ ]:
# Работа с табличными данными (DataFrame)
import pandas as pd  
# Пространственные данные на базе pandas (GeoDataFrame)        
import geopandas as gpd   
# Загрузка и анализ данных OpenStreetMap    
import osmnx as ox     
# Построение графиков и карт       
import matplotlib.pyplot as plt  
# Настройка стилей и колормэппингов для графиков
import matplotlib as mpl   
# Генерация Вороного-диаграмм по множеству точек
from shapely.ops import voronoi_diagram  
# Создание и работа с геометрическими фигурами
from shapely.geometry import MultiPoint  
# Работа с числовыми массивами и векторными операциями
import numpy as np  
# Вычисление попарных расстояний между точками (матрица расстояний)
from scipy.spatial.distance import cdist


## 1. Агрегирование атрибутивной информации


На прошлой неделе мы научились агрегировать точечный слой по полигонам — подсчитывать число точек. Сегодня разберёмся, как агрегировать атрибуты объектов


Прочитаем данные об МКД в Санкт-Петербурге и о границах районов


In [ ]:
#Данные с округами в Санкт-Петербурге в формате geopackage
admin_okrug = gpd.read_file('data/spb_admin.gpkg', layer="okrug")

#МКД Санкт-Петербурга в формате csv
mkd = pd.read_csv('data/spb_mkd.csv')
#Разбиваем поле с координатами на два - 'lat' -- широта; 'lon' -- долгота
mkd[['lat', 'lon']] = mkd['coordinates'].str.split(',', expand=True).astype(float)
#На основе DataFrame создаем GeoDataFrame, определяя геометрию на основе координат
mkd_gdf = gpd.GeoDataFrame(mkd, geometry=gpd.points_from_xy(mkd['lon'], mkd['lat']), crs=4326)

Давайте посмотрим, какая атрибутивная информация есть в слое с МКД


In [ ]:
# такая запись помогает нам не просто показать первые строки, но и транспонировать таблицу, чтобы увидеть сразу все столбцы
mkd_gdf.head().T

У нас, к сожалению, нет документации с описанием названий полей, но мы +- можем догадываться, какая именно информация содержится в полях.

Из интересной информации, которую бы мы хотели посмотреть по каждому из округов можно выбрать:
data_buildingdate - год постройки;

**Давайте для каждого района посчитаем средний, медианный, самый ранний и самый поздний год постройки МКД**


Первые шаги будут аналогичны тем, которые мы выполняли на прошлой неделе при подсчете кол-ва точек по ячейкам регулярной сетки


##### Шаг1: Выполняем пространственное объединение (Spatial Join)

Пересекаем точки с кафе и полигоны регулярной сетки (не забываем, что слои должны быть в одной системе координат)


In [ ]:
# sjoin: определяем, в какой округ попадает каждый из МКД
mkd_in_okrug = gpd.sjoin(mkd_gdf, admin_okrug, predicate="within", how="left")

mkd_in_okrug.head()

В атрибутивной таблице выше - все поля, которые идут после "geometry" -- атрибутивная информация из слоя с округами, в зависимости от того, в какой округ попадает та или иная точка


##### Шаг2: Рассчитываем статистику


Группируем данные на основе их попадания в тот или иной округ (например, по полю index_right) и рассчитываем значения

**Важно**: поскольку мы хотим рассчитывать количественную статистику, необходимо проверить типы данных полей
(data_buildingdate, data_residents) и убедиться, что они имеют числовой тип или преобразовать их в числовой


In [ ]:
# 1. Посмотреть исходные типы
print("До преобразования:")
print(mkd_in_okrug[['data_buildingdate']].dtypes)

# 2. Преобразование

mkd_in_okrug['data_buildingdate'] = pd.to_numeric(mkd_in_okrug['data_buildingdate'], errors='coerce')

# 3. Проверка — типы после
print("\nПосле преобразования:")
print(mkd_in_okrug['data_buildingdate'].dtypes)


Отлично! Теперь мы точно можем рассчитывать количественную статистику на основе этого поля


Рассчитаем средний год постройки МКД в разных округах


In [ ]:
build_year_mean = mkd_in_okrug.groupby('index_right')['data_buildingdate'].mean()

# Добавляем в данные поля с средним годом постройки в слой с округами
admin_okrug['build_year_mean'] = build_year_mean

# Смотрим на результат
admin_okrug.explore(column='build_year_mean', cmap='YlGnBu', tiles='CartoDB positron')



Выглядит очень интересно и правдоподобно :)


По аналогии посчитаем медианный год постройки, а также самый ранний и самый поздний


In [ ]:
build_year_median = mkd_in_okrug.groupby('index_right')['data_buildingdate'].median()
build_year_min = mkd_in_okrug.groupby('index_right')['data_buildingdate'].min()
build_year_max = mkd_in_okrug.groupby('index_right')['data_buildingdate'].max()

# Добавляем в данные соответсвующие поля
admin_okrug['build_year_median'] = build_year_median
admin_okrug['build_year_min'] = build_year_min
admin_okrug['build_year_max'] = build_year_max



Отобразим 4 показателя вместе для сравнения


In [ ]:
metrics = [
    ('build_year_mean', 'Средний год постройки'),
    ('build_year_median', 'Медианный год постройки'),
    ('build_year_min', 'Самый ранний год постройки'),
    ('build_year_max', 'Самый поздний год постройки'),
]

# Общий диапазон для всех карт
vmin = admin_okrug[[m for m, _ in metrics]].min().min()
vmax = admin_okrug[[m for m, _ in metrics]].max().max()

# Создаём одну фигуру с 1 строкой и 4 столбцами
fig, axes = plt.subplots(1, 4, figsize=(20, 5), constrained_layout=True)

# Создаём общий нормализатор и мэппер для цветовой шкалы
norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = mpl.cm.viridis  # или любая другая

for ax, (metric, title) in zip(axes, metrics):
    admin_okrug.plot(
        column=metric,
        ax=ax,
        cmap=cmap,
        norm=norm,
        linewidth=0.5,
        edgecolor='gray'
    )
    ax.set_title(title, fontsize=12)
    ax.axis('off')

# Общая colorbar
sm = mpl.cm.ScalarMappable(cmap=cmap, norm=norm)
sm._A = []  # пустой массив, чтобы colorbar работал
cbar = fig.colorbar(sm, ax=axes.tolist(), orientation='horizontal',
                    fraction=0.05, pad=0.02)
cbar.set_label('Год постройки', fontsize=12)

plt.show()

Красота!


## 2. Построение диаграммы Вороного


Диаграмма Вороного (или разбиение Вороного) — это разбиение пространства на ячейки по следующему правилу: каждой точке-генератору (site) сопоставляется такая область, что любая точка внутри этой области ближе к нему, чем к любой другой.


Загрузим данные станций метро и разобьём территорию Петербурга на области Вороного: каждая область соответствует ближайшей станции


In [ ]:
#Прочитаем данные о станциях метро
metro = gpd.read_file('data/spb_metro.geojson')

#Создадим полигон с границами города на основе admin_okrug c помощью инструмента dissolve
admin_border = admin_okrug.dissolve()


# Построим карту с двумя слоями
fig, ax = plt.subplots(figsize=(8, 8))
# Сначала — граница города
admin_border.plot(ax=ax, color='none', edgecolor='black', linewidth=1)
# Затем — станции метро
metro.plot(ax=ax, color='red', markersize=10, label='Станции метро')
# Оформление
ax.set_title('Граница СПб и станции метро', fontsize=14)
ax.axis('off')
ax.legend()

plt.tight_layout()
plt.show()


Строим диаграмму Вороного


In [ ]:
# 1. Собираем все точки-генераторы в один объект MultiPoint
#    Здесь каждая станция метро — это точка (Point).
#    .geometry.tolist() возвращает список shapely-геометрий.
points_mp = MultiPoint(metro.geometry.tolist())

# 2. Генерируем диаграмму Вороного по точечному набору станций метро,
#    ограничивая все ячейки рамкой города (envelope) — чтобы полигоны
#    не выходили за пределы Санкт-Петербурга.
vor = voronoi_diagram(points_mp, envelope=admin_border.geometry.iloc[0])

# 3. Создаём GeoDataFrame из полигонов Вороного:
#    конвертируем коллекцию геометрий в список и передаём его в GeoDataFrame,
#    сохраняя исходную систему координат (CRS) станций метро.
cells = list(vor.geoms)
vor_gdf = gpd.GeoDataFrame(geometry=cells, crs=metro.crs)

# 4. Визуализация результата
fig, ax = plt.subplots(figsize=(8, 8))

# 4.1. Рисуем полигоны диаграммы Вороного
#      cmap='gist_ncar' — качественная палитра с множеством цветов.
#      edgecolor='white' и alpha=0.6 делают границы полигонов чёткими,
#      но сохраняют прозрачность для наглядности.
vor_gdf.plot(
    ax=ax,
    cmap='gist_ncar',
    edgecolor='white',
    alpha=0.6
)

# 4.2. Наносим станции метро поверх полигонов
#      color='red' и markersize задают красные точки нужного размера.
metro.plot(
    ax=ax,
    color='red',
    markersize=8,
    label='Станции метро'
)

# 4.3. Рисуем контур границы города
#      boundary.plot рисует только линию границы без заливки.
admin_border.boundary.plot(
    ax=ax,
    color='black',
    linewidth=1,
    label='Граница СПб'
)

# 5. Настройка оформления
ax.set_title(
    "Диаграмма Вороного по станциям метро СПб (Shapely)",
    fontsize=14
)
ax.axis('off')   # отключаем оси, они не нужны на карте
ax.legend()      # выводим легенду по точкам и границе

plt.tight_layout()
plt.show()


Обрежем диаграмму Вороного по границам города


In [ ]:
# 1.Выполняем обрезку (clip) полигонов Вороного по границе Санкт-Петербурга
clipped_vor = gpd.clip(vor_gdf, admin_border.geometry.unary_union)

# 2. Визуализируем результат
fig, ax = plt.subplots(figsize=(8, 8))

# 2.1. Рисуем обрезанные полигоны Вороного
clipped_vor.plot(
    ax=ax,
    cmap='gist_ncar',
    edgecolor='white',
    alpha=0.6
)

# 2.2. Наносим станции метро поверх полигонов
metro.plot(
    ax=ax,
    color='red',
    markersize=8,
    label='Станции метро'
)

# 2.3. Рисуем контур границы города
admin_border.boundary.plot(
    ax=ax,
    color='black',
    linewidth=1,
    label='Граница СПб'
)

# 3. Оформление
ax.set_title("Диаграмма Вороного по станциям метро СПб 2.0", fontsize=14)
ax.axis('off')
ax.legend()

plt.tight_layout()
plt.show()

Мы построили классическую диаграмму Вороного по станциям метро — пространство города разбилось на ячейки, каждая из которых показывает зону ближайшей станции. Это наглядно демонстрирует «зоны влияния» метро и помогает быстро оценить доступность станций для разных кварталов. Однако следует помнить, что вычисления основаны на прямом (евклидовом) расстоянии, без учёта реальной уличной сети, барьеров и особенностей рельефа. Поэтому, несмотря на полезную наглядность, для точного анализа доступности стоит использовать инструменты сетевого анализа (с некоторыми из них мы познакомимся в следующие недели)


## 3. Nearest Neighbour Index


NNI (Nearest Neighbor Index, индекс ближайшего соседа) — это статистическая мера, которая позволяет оценить, насколько равномерно распределены объекты в пространстве. Давайте посмотрим, на распределение театров в Санкт-Петербурге


Прочитаем данные о театрах и подготовим их для последующего анализа


In [ ]:
theaters = gpd.read_file('data/spb_theaters.geojson')
theaters = theaters.dropna(subset=['geometry'])  # убираем точки без геометрии

# Перепрецируем в подходящую UTM-зону
utm_crs = theaters.estimate_utm_crs()
theaters_utm = theaters.to_crs(utm_crs)

# Преобразуем геометрические данные в координаты
theaters_coords = np.array([point.coords[0] for point in theaters_utm.geometry])

Производим необходимые вычисления


In [ ]:
# 1. Построим матрицу евклидовых расстояний и сразу «забаним» диагональ
dist_matrix = cdist(theaters_coords, theaters_coords, metric='euclidean')
np.fill_diagonal(dist_matrix, np.inf)
nearest_neighbor_distances = dist_matrix.min(axis=1)

# 2. Среднее расстояние до ближайшего соседа
average_nearest_neighbor_distance = nearest_neighbor_distances.mean()

# 3. Площадь Санкт-Петербурга в той же метрической CRS
admin_border_utm = admin_border.to_crs(utm_crs)
admin_area = admin_border_utm.geometry.area.iloc[0]

# 4. Ожидаемое среднее при случайном распределении
n = len(theaters_coords)
expected_mean_distance = 0.5 * np.sqrt(admin_area / n)

# 5. Рассчитываем NNI
NNI = average_nearest_neighbor_distance / expected_mean_distance
print("NNI:", NNI)


Полученное значение **NNI ≈ 0.43** существенно меньше единицы. Это говорит о том, что распределение театров в Петербурге **кластери­зовано**, а не случайно или равномерно.


## 4.Итог


1. Мы научились агрегировать пространственные и атрибутивные данные: считали средний, минимальный и максимальный годы постройки МКД по округам Санкт-Петербурга

2. Познакомились с Диаграммой Вороного и построили «зоны влияния» станций метро: каждая точка города отнесена к ближайшей станции по евклидову расстоянию.

3. Узнали об оценке кластеризации точечных объектов с помощью Nearest Neighbour Index.

При этом всегда важно помнить ограничения методов: диаграмма Вороного и NNI опираются на прямое (евклидово) расстояние и не учитывают реальную уличную сеть и препятствия. Для более точных пространственных моделей стоит применять сетевой анализ (с которым мы познакомимся на следующих занятиях).

На основе материалов этого и [прошлого](./week2.ipynb) занятия вы сможете полностью выполнить [второй проект](../projects/project2.ipynb)
